# Financial Coaching Agent using PraisonAIAgents

This notebook demonstrates how to build a Financial Coaching Agent using the `praisonaiagents` library.
The agent analyzes user financial data and provides actionable recommendations for budgeting, savings, and debt management.
The notebook is CI-friendly: it uses dummy data if API keys are not set, so all tests/checks will pass.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/Finance_Market_Job_Agents/Financial_Coaching_Agent.ipynb)


# Dependencies

In [ ]:
!pip install praisonaiagents pandas

# Set up key

In [2]:
import os

# Try to get keys from environment variables, or use placeholders
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "sk-..")

# Set them in the environment for downstream code
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

def is_valid_key(key, prefix):
    return key and key != f"{prefix}-.." and key.startswith(prefix)

# Tool: Financial Summary

In [3]:
import pandas as pd

class FinancialSummaryTool:
    """
    Custom tool to summarize financial data and calculate basic stats.
    """
    def __init__(self, income, expenses, debts):
        self.income = income
        self.expenses = expenses
        self.debts = debts

    def summary(self):
        total_expenses = sum(self.expenses.values())
        surplus = self.income - total_expenses
        total_debt = sum(d['amount'] for d in self.debts)
        return {
            "total_expenses": total_expenses,
            "surplus": surplus,
            "total_debt": total_debt,
            "expense_breakdown": self.expenses,
            "debts": self.debts
        }

# YAML prompt

In [4]:
import yaml

yaml_prompt = """
agent:
  name: Financial Coaching Agent
  description: |
    An agent that analyzes user financial data and provides actionable recommendations for budgeting, savings, and debt management.
  tools:
    - name: FinancialSummaryTool
      description: Summarizes income, expenses, and debts for the agent.
  workflow:
    - step: Summarize financial data using FinancialSummaryTool
    - step: Analyze summary and provide recommendations
inputs:
  - monthly_income
  - expenses
  - debts
outputs:
  - recommendations
"""
print(yaml.safe_load(yaml_prompt))

{'agent': {'name': 'Financial Coaching Agent', 'description': 'An agent that analyzes user financial data and provides actionable recommendations for budgeting, savings, and debt management.\n', 'tools': [{'name': 'FinancialSummaryTool', 'description': 'Summarizes income, expenses, and debts for the agent.'}], 'workflow': [{'step': 'Summarize financial data using FinancialSummaryTool'}, {'step': 'Analyze summary and provide recommendations'}]}, 'inputs': ['monthly_income', 'expenses', 'debts'], 'outputs': ['recommendations']}


# Main

In [9]:
from praisonaiagents import Agent

# Dummy data for CI/public use
dummy_financial_data = {
    "monthly_income": 3000,
    "expenses": {
        "Housing": 1200,
        "Food": 400,
        "Transportation": 200,
        "Entertainment": 150,
        "Savings": 300,
        "Other": 100
    },
    "debts": [
        {"name": "Credit Card", "amount": 2000, "interest_rate": 18.0, "min_payment": 50},
        {"name": "Student Loan", "amount": 8000, "interest_rate": 5.0, "min_payment": 100}
    ]
}

income = dummy_financial_data["monthly_income"]
expenses = dummy_financial_data["expenses"]
debts = dummy_financial_data["debts"]

tool = FinancialSummaryTool(income, expenses, debts)
summary = tool.summary()

if not is_valid_key(OPENAI_API_KEY, "sk"):
    print("API key not set or is a placeholder. Using dummy recommendations for CI/testing.")
    recommendations = """
    - Review your monthly expenses and identify areas to cut back.
    - Increase your monthly savings by at least 10%.
    - Pay off high-interest debts first (e.g., Credit Card).
    - Build an emergency fund covering at least 3-6 months of expenses.
    """
else:
    prompt = f"""
    You are a financial coach. Analyze the following summary and provide actionable recommendations:
    - Monthly income: ${summary['surplus'] + summary['total_expenses']}
    - Total expenses: ${summary['total_expenses']}
    - Surplus: ${summary['surplus']}
    - Total debt: ${summary['total_debt']}
    - Expense breakdown: {summary['expense_breakdown']}
    - Debts: {summary['debts']}
    """
    agent = Agent(
        name="Financial Coaching Agent",
        api_key=OPENAI_API_KEY
    )
    # Use start(prompt) instead of run()
    recommendations = agent.start(prompt)

print("=== Financial Coaching Agent Recommendations ===")
print(recommendations)

╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Financial Coaching Agent                                                                             │
│  Role: Assistant                                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Response generated in 11.0s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ You are a financial coach. Analyze the following summary and provide actionable recommendations: - Monthly      │
│ income: $3000 - Total expenses: $2350 - Surplus: $650 - Total debt: $10000 - Expense breakdown: {'Housing':     │
│ 1200, 'Food': 400, 'Transportation': 200, 'Entertainment': 150, 'Savings': 300, 'Other': 100} - Debts:          │
│ [{'name': 'Credit Card', 'amount': 2000, 'interest_rate': 18.0, 'min_payment': 50}, {'name': 'Student Loan',    │
│ 'amount': 8000, 'interest_rate': 5.0, 'min_payment': 100}]                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│ Based on the provided financial summary, here are some actionable recommendations to help manage your finances  │
│ more effectively and reduce your debt:                                                                          │
│                                                                                                                 │
│  1 Prioritize Debt Repayment:                                                                                   │
│     • Focus on paying off the high-interest debt first, which is the credit card debt with an 18% interest      │
│       rate. Consider using the debt avalanche method, where you pay extra towards the debt with the highest     │
│       interest rate while making minimum payments on others.                                                    │
│     • With a surplus of $650, allocate a significant portion of this towards the credit card debt. For example, │
│       you could pay $400 towards the credit card debt (including the $50 minimum payment), which would help     │
│       reduce the principal faster and save on interest.                                                         │
│  2 Review and Adjust Expenses:                                                                                  │
│     • Housing: At $1200, housing is a significant portion of your expenses. If possible, explore options to     │
│       reduce this cost, such as negotiating rent, finding a roommate, or moving to a less expensive place.      │
│     • Food: Consider meal planning, cooking at home, and reducing dining out to lower the $400 food expense.    │
│     • Entertainment: At $150, see if you can cut back on discretionary spending. Look for free or low-cost      │
│       entertainment options.                                                                                    │
│     • Other Expenses: Review the $100 in 'Other' expenses to identify any non-essential spending that can be    │
│       reduced or eliminated.                                                                                    │
│  3 Increase Savings:                                                                                            │
│     • While you are already saving $300 monthly, consider increasing this amount once the high-interest debt is │
│       paid off. Building an emergency fund should be a priority to cover 3-6 months of expenses.                │
│  4 Boost Income:                                                                                                │
│     • Explore opportunities to increase your income, such as taking on a part-time job, freelancing, or selling │
│       unused items. Any additional income can accelerate debt repayment and increase savings.                   │
│  5 Monitor and Adjust:                                                                                          │
│     • Regularly review your budget and expenses to ensure you are on track with your financial goals. Adjust    │
│       your plan as needed to accommodate changes in income or expenses.                                         │
│  6 Consider Debt Consolidation:                                                                                 │
│     • If you have a good credit score, look into consolidating your credit card debt into a lower-interest      │
│       personal loan. This could reduce your overall interest payments and simplify your debt repayment process. │
│                                                                                                                 │
│ By implementing these recommendations, you can effectively manage your finances, reduce debt, and work towards  │
│ achieving greater financial stability.                                                                          │
╰───────────────────────────────────────────────────────

=== Financial Coaching Agent Recommendations ===
Based on the provided financial summary, here are some actionable recommendations to help manage your finances more effectively and reduce your debt:

1. **Prioritize Debt Repayment:**
   - Focus on paying off the high-interest debt first, which is the credit card debt with an 18% interest rate. Consider using the debt avalanche method, where you pay extra towards the debt with the highest interest rate while making minimum payments on others.
   - With a surplus of $650, allocate a significant portion of this towards the credit card debt. For example, you could pay $400 towards the credit card debt (including the $50 minimum payment), which would help reduce the principal faster and save on interest.

2. **Review and Adjust Expenses:**
   - **Housing:** At $1200, housing is a significant portion of your expenses. If possible, explore options to reduce this cost, such as negotiating rent, finding a roommate, or moving to a less expensive